## **Introduction to DARLi, Basics of Backends and Models** 


The DARLi (Differentiable Articulated Robotics Library) is a wrapper around the CasADi kinematics/dynamics and Pinocchio libraries. Its primary goal is to facilitate the creation of differentiable models for robotic systems, provided a URDF (Unified Robot Description Format) file. **DARLi is not an implementation of mechanics-oriented** algorithms, such as Featherstone's Articulated Body or the Recursive Newton-Euler. Instead, we rely on the efficient implementations provided by [Pinocchio](https://github.com/stack-of-tasks/pinocchio/tree/master) and offer a wrapper that grants easy access to a suite of features that we and our colleagues have found useful in practice of trajectory planning, feedback control, and system identification over poly-articulated robots.

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

**Note**: You have to restart kernel after `robot_descriptions` is installed

In [1]:

%%capture
!pip3 install robot_descriptions
from robot_descriptions import iiwa7_description

### Backends 
The typical DARLi model is based on instance of Backend. Backend take `urdf` file as input and defines the way of computing two main functions: Articulated Bpdy Algorithm `aba` and Recursive Newton Euler `rnea`, as well as severals utility functions. These are soledly based on Pinnochio. For now there are two backends are available:

TODO: WHAT IS RNEA AND ABA

- `PinocchioBackend` - Provide interface to the Pinnochio python bindings. And may be useufull if one wants to obtain fast **numericall** computations. 
- `CasadiBackend` - The CasADi wrapper around Pinnochiio, this is used to cimpute dynamics in **symbolical** fashion, this allows for seeamless calculation of derivative and potentially used in trajectory optimization and optimal control.


The acces to backends is as simple as:

In [2]:
from darli.backend import CasadiBackend, PinocchioBackend

# Building the Pinnochio backend
pin_backend = PinocchioBackend(iiwa7_description.URDF_PATH)
# Building the CasADi backend
cas_backend = CasadiBackend(iiwa7_description.URDF_PATH)

In [3]:
pin_backend.nq
pin_backend.nv
pin_backend.aba
pin_backend.rnea

<bound method PinocchioBackend.rnea of <darli.backend._pinocchio.PinocchioBackend object at 0x7c9599784a30>>

### Floating Base, Fixed joints and Conventions: 



TODO: Something on floating base and conventions

In [4]:
from robot_descriptions import go2_description

In [5]:
go2_cas = CasadiBackend(go2_description.URDF_PATH)

In [6]:
print(f"Dimensions of model, nq: {go2_cas.nq} nv: {go2_cas.nv}")

Dimensions of model, nq: 12 nv: 12


In [7]:
from darli.backend import JointType

go2_cas_floating = CasadiBackend(
    go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER
)

print(
    f"Dimensions of floating model, nq: {go2_cas_floating.nq} nv: {go2_cas_floating.nv}"
)

Dimensions of floating model, nq: 19 nv: 18


In [8]:
cas_backend = CasadiBackend(go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER)

The model in DARLi is a building block that can be further extended with external forces and contacts, different selector matrices, friction in joints, state space and parametric representations, please follow up to the next steps of tutorial to get in to that.

- General-info
- Backends
- Models 
- Floating Base,Conventions 
- Further Read and Structure


- Model of Poly-Articulated system
- Conventions
- rnea and aba
- Floating Base


### Conventions



### Backends

In [9]:
from darli.model import Model

In [10]:
casadi_model = Model(CasadiBackend(iiwa7_description.URDF_PATH))

In [11]:
casadi_model.com()

CoM(position=SX(@1=cos(q_0), @2=-2.46519e-32, @3=sin(q_1), @4=cos(q_1), @5=sin(q_2), @6=cos(q_2), @7=cos(q_3), @8=-1.22465e-16, @9=sin(q_4), @10=cos(q_4), @11=1.54272e-33, @12=cos(q_5), @13=0.317437, @14=sin(q_5), @15=(0.00021633+((@11*@12)-(@13*@14))), @16=2.22045e-16, @17=(0.0454293+(((@11*(@16*@14))+((@13*(@16*@12))+-0.190869))+0.332369)), @18=(0.164411+(((@11*@14)+((@13*@12)+4.23814e-17))+1.04036)), @19=((((@8*@9)-@10)*@15)+(((@9+(@8*@10))*@17)+(2.46519e-32*@18))), @20=sin(q_3), @21=(0.233307+((((@16*@9)*@15)+(((@16*@10)*@17)+@18))+1.60417)), @22=((@7*@19)-(@20*@21)), @23=-2.22045e-16, @24=(0.118395+((((@8*@10)+@9)*@15)+(((@10-(@8*@9))*@17)+(@23*@18)))), @25=(0.121687+(((@16*@20)*@19)+(((@16*@7)*@21)-@24))), @26=1.22465e-16, @27=(0.52731+(((@20*@19)+((@7*@21)+(@16*@24)))+2.11301)), @28=(0.00104463+((((@2*@5)-@6)*@22)+(((@5+(@2*@6))*@25)+(@26*@27)))), @29=(0.205444+(((((@26*@6)+(@23*@5))*@22)+((((@23*@6)-(@26*@5))*@25)+@27))+3.18724)), @30=(0.146248+((@5*@22)+((@6*@25)+(@16*@27)))),

One can retrive basic model info as follows:

In [12]:
nq = casadi_model.nq  # dimensionality of configuration
nv = casadi_model.nv  # dimensionality of generilized velocities
nu = casadi_model.nu  # dimensionality  of control inputs
q_min, q_max = (
    casadi_model.q_min,
    casadi_model.q_max,
)  # minimal and maximal limits on joint pos
nq, nv, nu

(7, 7, 7)

In [13]:
joint_names = casadi_model.joint_names  # names of the joints
joint_names

['universe',
 'iiwa_joint_1',
 'iiwa_joint_2',
 'iiwa_joint_3',
 'iiwa_joint_4',
 'iiwa_joint_5',
 'iiwa_joint_6',
 'iiwa_joint_7']

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [14]:
inertia = casadi_model.inertia
gravity_vector = casadi_model.gravity
bias_force = casadi_model.bias_force
coriolis_matrix = casadi_model.coriolis_matrix
coriolis = casadi_model.coriolis

Each of the above define the CasAdi functions:

And can be evaluated both numerically and Functionalally:

In [15]:
inertia = casadi_model.inertia
import numpy as np
import casadi as cs

# Functionalal computation
print("Functional:", inertia(cs.SX.sym("q", nq)))
# Numerical computation
# print("Numerical:", inertia(np.random.randn(nq)))

Functional: @1=sin(q_1), @2=0.011768, @3=2.22045e-16, @4=sin(q_6), @5=(@3*@4), @6=0.0005912, @7=cos(q_6), @8=(@3*@7), @9=((@2*@5)+(@6*@8)), @10=0.011778, @11=((@6*@5)+(@10*@8)), @12=((@9*@5)+(@11*@8)), @13=-1.22465e-16, @14=((@13*@7)+@4), @15=(@7-(@13*@4)), @16=((@2*@14)+(@6*@15)), @17=((@6*@14)+(@10*@15)), @18=((@16*@14)+(@17*@15)), @19=0.002872, @20=((0.004527+(@18+@19))+0.013517), @21=(((0.006509+(((0.023546-@12)-@18)+@19))+0.0183928)-@20), @22=(((0.006259+(@12+@19))+0.00487582)-@20), @23=sin(q_5), @24=(@3*@23), @25=cos(q_5), @26=(@3*@25), @27=((@13*@4)-@7), @28=(@4+(@13*@7)), @29=(((@9*@27)+(@11*@28))-6.63784e-35), @30=2, @31=(((@16*@27)+(@17*@28))-3.98667e-35), @32=(@30*@31), @33=((@24*@21)+((@26*@29)-@32)), @34=((0.00031891+((@16*@5)+(@17*@8)))-0.00811828), @35=(@34+@34), @36=((@24*@29)+((@26*@22)-@35)), @37=((@33*@24)+(@36*@26)), @38=((@23*@21)+((@25*@29)+(@3*@32))), @39=((@23*@29)+((@25*@22)+(@3*@35))), @40=((@38*@23)+(@39*@25)), @41=1.55066, @42=0.0607, @43=2.81744e-34, @44=0.

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [16]:
casadi_model.energy()

Energy(kinetic=SX(@1=-0.03, @2=(sin(q_1)*v_0), @3=-3.4821, @4=0.059, @5=0.0003, @6=(cos(q_1)*v_0), @7=((@5*@6)-(@4*@2)), @8=0.042, @9=2.22045e-16, @10=(v_1+(@9*v_0)), @11=((@8*@2)-(@5*@10)), @12=((@4*@10)-(@8*@6)), @13=2, @14=sin(q_2), @15=0.21, @16=(@15*@2), @17=-2.46519e-32, @18=cos(q_2), @19=((@17*@14)-@18), @20=(@15*@10), @21=((@14*@16)-(@19*@20)), @22=(@14+(@17*@18)), @23=((@18*@16)-(@22*@20)), @24=1.22465e-16, @25=((@9*@16)-(@24*@20)), @26=0.03, @27=(v_2+((@24*@2)+(@6+(@9*@10)))), @28=0.13, @29=-2.22045e-16, @30=((@22*@2)+((((@29*@18)-(@24*@14))*@6)+(@18*@10))), @31=((@26*@27)-(@28*@30)), @32=((@19*@2)+((((@24*@18)+(@29*@14))*@6)+(@14*@10))), @33=(@28*@32), @34=(@26*@32), @35=-4.05623, @36=cos(q_3), @37=0.19, @38=(@21+(@37*@30)), @39=sin(q_3), @40=(@9*@39), @41=(@23-(@37*@32)), @42=((@36*@38)+((@40*@41)+(@39*@25))), @43=(@9*@36), @44=(((@43*@41)+(@36*@25))-(@39*@38)), @45=((@9*@25)-@41), @46=0.067, @47=(v_3+((@9*@27)-@30)), @48=0.034, @49=(((@43*@30)+(@36*@27))-(@39*@32)), @50=((

In [17]:
com_position = casadi_model.com().position
com_velocity = casadi_model.com().velocity
com_jacobian = casadi_model.com().jacobian
com_jacobian_dt = casadi_model.com().jacobian_dt
potential_energy = casadi_model.energy().potential
kinetic_energy = casadi_model.energy().kinetic
lagrangian = casadi_model.lagrangian()

# TODO
# momentum

In [18]:
com_jacobian_dt

SX(@1=2.22045e-16, @2=3.4821, @3=0.042, @4=cos(q_1), @5=(@4*v_0), @6=0.059, @7=(v_1+(@1*v_0)), @8=(@2*((@3*@5)-(@6*@7))), @9=-2.22045e-16, @10=1.22465e-16, @11=sin(q_1), @12=((@9*@4)-(@10*@11)), @13=sin(q_0), @14=(-0.0452245*@13), @15=-2.46519e-32, @16=(@11+(@15*@4)), @17=0.0452245, @18=cos(q_0), @19=(@17*@18), @20=((@12*@14)+(@16*@19)), @21=(@14+(@10*@19)), @22=(@2*((@7*@20)-(@5*@21))), @23=4.05623, @24=0.13, @25=sin(q_2), @26=cos(q_2), @27=(@25+(@15*@26)), @28=(@11*v_0), @29=((@9*@26)-(@10*@25)), @30=((@27*@28)+((@29*@5)+(@26*@7))), @31=0.03, @32=(v_2+((@10*@28)+(@5+(@1*@7)))), @33=0.21, @34=(@33*@28), @35=((@15*@25)-@26), @36=(@33*@7), @37=((@25*@34)-(@35*@36)), @38=(@23*(((@24*@30)-(@31*@32))+@37)), @39=((@10*@4)+(@9*@11)), @40=((@15*@11)-@4), @41=((@39*@14)+(@40*@19)), @42=(((@26*@21)+(@29*@20))+(@27*@41)), @43=(((@1*@21)+@20)+(@10*@41)), @44=(@23*((@32*@42)-(@30*@43))), @45=3.4822, @46=0.034, @47=cos(q_3), @48=(@1*@47), @49=sin(q_3), @50=((@10*@26)+(@9*@25)), @51=((@35*@28)+((@50

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [19]:
casadi_model.contact_qforce

SX(@1=0, [@1, @1, @1, @1, @1, @1, @1])

In [20]:
casadi_model.inverse_dynamics()

SX(@1=-0.03, @2=sin(q_1), @3=0.02076, @4=cos(q_1), @5=(@4*v_0), @6=((v_1*@5)+(@2*dv_0)), @7=-0.003626, @8=2.22045e-16, @9=(dv_1+(@8*dv_0)), @10=0.059, @11=3.4821, @12=2.17826e-15, @13=0.0003, @14=(@2*v_0), @15=((@4*dv_0)-(v_1*@14)), @16=(@11*(@12-((@13*@15)-(@10*@6)))), @17=0.042, @18=9.81, @19=(@18*@4), @20=(@11*(@19-((@17*@6)-(@13*@9)))), @21=0.00779, @22=(v_1+(@8*v_0)), @23=(@11*((@10*@22)-(@17*@5))), @24=(@11*((@17*@14)-(@13*@22))), @25=(((@7*@14)+(@21*@22))+((@10*@23)-(@13*@24))), @26=0.02179, @27=(@11*((@13*@5)-(@10*@14))), @28=((@26*@5)+((@13*@27)-(@17*@23))), @29=-2.46519e-32, @30=sin(q_2), @31=cos(q_2), @32=((@29*@30)-@31), @33=0.03204, @34=(@30+(@29*@31)), @35=-2.22045e-16, @36=1.22465e-16, @37=((@35*@31)-(@36*@30)), @38=((@34*@14)+((@37*@5)+(@31*@22))), @39=((@36*@31)+(@35*@30)), @40=((v_2*@38)+((@32*@6)+((@39*@15)+(@30*@9)))), @41=0.03, @42=4.05623, @43=(@18*@2), @44=0.21, @45=(@43-(@44*@9)), @46=(@12+(@44*@6)), @47=((@36*@45)+(@19+(@8*@46))), @48=(@42*(@47+(@41*@40))), @49

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [21]:
casadi_model.contact_qforce

SX(@1=0, [@1, @1, @1, @1, @1, @1, @1])

In [22]:
casadi_model.forward_dynamics()

SX(@1=sin(q_1), @2=cos(q_1), @3=(@2*v_0), @4=-0.003626, @5=(@1*v_0), @6=2.22045e-16, @7=(v_1+(@6*v_0)), @8=0.059, @9=3.4821, @10=0.042, @11=(@9*((@8*@7)-(@10*@3))), @12=0.0003, @13=(@9*((@10*@5)-(@12*@7))), @14=(((@4*@5)+(0.00779*@7))+((@8*@11)-(@12*@13))), @15=(@9*((@12*@3)-(@8*@5))), @16=((0.02179*@3)+((@12*@15)-(@10*@11))), @17=-2.46519e-32, @18=sin(q_2), @19=cos(q_2), @20=((@17*@18)-@19), @21=(@18+(@17*@19)), @22=-2.22045e-16, @23=1.22465e-16, @24=((@22*@19)-(@23*@18)), @25=((@21*@5)+((@24*@3)+(@19*@7))), @26=0.006227, @27=(v_2+((@23*@5)+(@3+(@6*@7)))), @28=0.03, @29=4.05623, @30=0.21, @31=(@30*@5), @32=(@30*@7), @33=((@18*@31)-(@20*@32)), @34=0.13, @35=(@29*(@33-((@28*@27)-(@34*@25)))), @36=(((@26*@25)+(0.03042*@27))-(@28*@35)), @37=(((0.00972*@25)+(@26*@27))+(@34*@35)), @38=((@19*@31)-(@21*@32)), @39=((@6*@31)-(@23*@32)), @40=((@23*@19)+(@22*@18)), @41=((@20*@5)+((@40*@3)+(@18*@7))), @42=(@29*(@39+(@28*@41))), @43=(@29*(@38-(@34*@41))), @44=cos(q_3), @45=(@6*@44), @46=sin(q_3), @

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [24]:
import numpy as np

print(f"Old input dimensions: {casadi_model.nu}")
S = np.random.randn(casadi_model.nv, casadi_model.nv + 2)
casadi_model.update_selector(S)
print(f"New input dimensions: {casadi_model.nu}")

Old input dimensions: 7
New input dimensions: 9


In [25]:
casadi_model.qfrc_u

SX([tau_0, tau_1, tau_2, tau_3, tau_4, tau_5, tau_6, tau_7, tau_8])

In [28]:
casadi_model.forward_dynamics

<bound method Model.forward_dynamics of <darli.model._model.Model object at 0x7c9590132ec0>>

In [29]:
casadi_model.update_selector(passive_joints=range(3))
print(f"New input dimensions: {casadi_model.nu}")
print(f"New selector:\n {casadi_model.selector}")

New input dimensions: 4
New selector:
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [30]:
casadi_model.forward_dynamics

<bound method Model.forward_dynamics of <darli.model._model.Model object at 0x7c9590132ec0>>

#### **Bodies and Contacts**

In [ ]:
model.add_body(["link06"])
model.bodies

In [ ]:
model.body("link06")

### Frames

In [ ]:
model.body("link06").linear_acceleration

In [ ]:
model.body("link06").linear_acceleration.local

In [ ]:
model.body("link06").jacobian.local

In [ ]:
model.body("link03").jacobian.local
model.body("link03").jacobian_dt.local
model.body("link03").linear_velocity.local
model.body("link03").angular_velocity.local
model.body("link03").linear_acceleration.local
model.body("link03").angular_acceleration.local

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

In [ ]:
model.state_space.state_jacobian

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

In [ ]:
model.body("link06").contact.cone.linear()

In [ ]:
model.add_body({"ee": "link06"})
model.bodies

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.